<a href="https://colab.research.google.com/github/Francho03/Praca-licencjacka/blob/main/Sprawdzenie_zbioru_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:


file_path = "/content/drive/MyDrive/praca_licencjacka/Sprawozdanie_przemoc_2014_2023.xlsx"
columns = ["lp", "woj", "pow", "gm", "typ_gm", "typ_jednostki", "keso", "rok", "liczba_proc_nk_a", "rodziny_wszczete", "liczba_osob_ogol", "liczba_osob_kobiety", "liczba_osob_mezczyzni", "liczba_osob_dzieci"]

dane = pd.read_excel(file_path, names=columns, engine='openpyxl')

dane["keso"] = dane["keso"].astype(str).str.extract(r"(\d{12,})")
dane.head()







,lp,woj,pow,gm,typ_gm,typ_jednostki,keso,rok,liczba_proc_nk_a,rodziny_wszczete,liczba_osob_ogol,liczba_osob_kobiety,liczba_osob_mezczyzni,liczba_osob_dzieci
0,1.0,NaN,NaN,NaN,NaN,MSW,NaN,2023,62170,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,MZ,NaN,2023,1902,NaN,NaN,NaN,NaN,NaN
2,3.0,dolnośląskie,NaN,NaN,NaN,WPS,100200000000,2023,1278,9848.0,15224.0,7780.0,1962.0,5482.0
3,4.0,dolnośląskie,bolesławiecki,Bolesławiec,miejska,OPS,300201011000,2023,24,194.0,223.0,161.0,36.0,26.0
4,5.0,dolnośląskie,bolesławiecki,Bolesławiec,wiejska,OPS,300201022000,2023,5,42.0,70.0,35.0,7.0,28.0


In [5]:
print(dane["rok"].value_counts())

rok
2022    2830
2023    2829
2014    2829
2016    2828
2015    2828
2018    2827
2017    2827
2020    2826
2019    2826
2021    2516
Name: count, dtype: int64


In [6]:

filtered_data = dane[((dane["woj"].isna()) & (dane["rok"].isin(range(2021, 2024)))) |
                     ((dane["woj"] == "RAZEM") & ~(dane["rok"].isin(range(2021, 2024))))]

print(filtered_data.groupby(["typ_jednostki", "rok"]).agg({"liczba_proc_nk_a": "sum"}))

                    liczba_proc_nk_a
typ_jednostki rok                   
MEiN          2021              3198
              2022              2838
              2023              4078
MKiDN         2021                33
              2022                42
MSW           2021             64250
              2022             61645
              2023             62170
MZ            2021              1138
              2022              2918
              2023              1902


In [7]:
filtered_gm_data = dane[dane["gm"].notna()]
print(filtered_gm_data.groupby("rok")["liczba_proc_nk_a"].sum())

rok
2014    13044
2015    12371
2016    11426
2017    13312
2018    10872
2019    11760
2020     9840
2021    60164
2022    14117
2023    17383
Name: liczba_proc_nk_a, dtype: int64


In [8]:
razem=dane[dane["woj"] == "RAZEM"]
razem=razem.iloc[:,[1,7,8,9,10,11,12,13]]
razem

,woj,rok,liczba_proc_nk_a,rodziny_wszczete,liczba_osob_ogol,liczba_osob_kobiety,liczba_osob_mezczyzni,liczba_osob_dzieci
2828,RAZEM,2023,85865,101785.0,163507.0,83026.0,18752.0,61729.0
5658,RAZEM,2022,83196,92789.0,140087.0,75512.0,16901.0,47674.0
8174,RAZEM,2021,128783,101292.0,161575.0,86995.0,23006.0,51574.0
11000,RAZEM,2020,10065,71719.0,156473.0,82643.0,27976.0,45854.0
13826,RAZEM,2019,12083,75234.0,164120.0,87121.0,31211.0,45788.0
16653,RAZEM,2018,11177,73760.0,162494.0,86281.0,28147.0,48066.0
19480,RAZEM,2017,13667,74850.0,160755.0,87681.0,26944.0,46130.0
22308,RAZEM,2016,11789,72346.0,162096.0,88106.0,30565.0,43425.0
25136,RAZEM,2015,12743,74277.0,149096.0,87460.0,26476.0,35160.0
27965,RAZEM,2014,13606,72692.0,146447.0,NaN,NaN,NaN


In [9]:
dane_razem_2021_2023 = dane[
    dane["typ_jednostki"].isin(["OPS", "WPS", "PCPR"]) &
    ~(dane["rok"].isin(range(2021, 2024)))]

print(dane_razem_2021_2023.groupby(["rok","typ_jednostki"]).agg({"liczba_proc_nk_a": "sum"}))


                    liczba_proc_nk_a
rok  typ_jednostki                  
2014 OPS                        9893
     PCPR                        562
     WPS                       13606
2015 OPS                        9399
     PCPR                        372
     WPS                       12743
2016 OPS                        8344
     PCPR                        363
     WPS                       11789
2017 OPS                        8537
     PCPR                        355
     WPS                       13667
2018 OPS                        7803
     PCPR                        305
     WPS                       11177
2019 OPS                        8559
     PCPR                        323
     WPS                       12083
2020 OPS                        7194
     PCPR                        225
     WPS                       10065


Dane z MSW, MEiN i MZ były zbierane tylko w latach 2021-2023 i to są dane przedstawiające liczbę procedur w całej polsce.

Natomiast dane z  MKiDN zostały zebrane tylko w latach 2021 oraz 2022, również dla całej polski.

In [10]:
wystapienia = dane.groupby(["rok", "typ_jednostki"])["liczba_proc_nk_a"].count()
suma = dane.groupby(["rok", "typ_jednostki"])["liczba_proc_nk_a"].sum()

typy_jednostek_liczba_proc_nk_a = pd.DataFrame({
    "ilosc_wystapien": wystapienia,
    "suma": suma
}).reset_index()

In [11]:
typy_jednostek_liczba_proc_nk_a

,rok,typ_jednostki,ilosc_wystapien,suma
0,2014,MOPR,65,3151
1,2014,OPS,2432,9893
2,2014,PCPR,315,562
3,2014,WPS,16,13606
4,2015,MOPR,65,2972
5,2015,OPS,2431,9399
6,2015,PCPR,315,372
7,2015,WPS,16,12743
8,2016,MOPR,65,3082
9,2016,OPS,2431,8344


In [12]:
pivot_typy_jednostek_liczba_proc_nk_a_ilosc_wystapien = typy_jednostek_liczba_proc_nk_a.pivot(index="rok", columns="typ_jednostki", values="ilosc_wystapien")
pivot_typy_jednostek_liczba_proc_nk_a_ilosc_wystapien = pivot_typy_jednostek_liczba_proc_nk_a_ilosc_wystapien.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_proc_nk_a_ilosc_wystapien

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,65,<NA>,<NA>,2432,315,16
2015,<NA>,<NA>,65,<NA>,<NA>,2431,315,16
2016,<NA>,<NA>,65,<NA>,<NA>,2431,315,16
2017,<NA>,<NA>,66,<NA>,<NA>,2430,314,16
2018,<NA>,<NA>,66,<NA>,<NA>,2430,314,16
2019,<NA>,<NA>,66,<NA>,<NA>,2429,314,16
2020,<NA>,<NA>,66,<NA>,<NA>,2429,314,16
2021,1,1,66,1,1,2429,<NA>,16
2022,1,1,66,1,1,2429,314,16


In [13]:
pivot_typy_jednostek_liczba_proc_nk_a_suma = typy_jednostek_liczba_proc_nk_a.pivot(index="rok", columns="typ_jednostki", values="suma")
pivot_typy_jednostek_liczba_proc_nk_a_suma = pivot_typy_jednostek_liczba_proc_nk_a_suma.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_proc_nk_a_suma

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,3151,<NA>,<NA>,9893,562,13606
2015,<NA>,<NA>,2972,<NA>,<NA>,9399,372,12743
2016,<NA>,<NA>,3082,<NA>,<NA>,8344,363,11789
2017,<NA>,<NA>,4775,<NA>,<NA>,8537,355,13667
2018,<NA>,<NA>,3069,<NA>,<NA>,7803,305,11177
2019,<NA>,<NA>,3201,<NA>,<NA>,8559,323,12083
2020,<NA>,<NA>,2646,<NA>,<NA>,7194,225,10065
2021,3198,33,16295,64250,1138,43869,<NA>,60164
2022,2838,42,3320,61645,2918,10797,328,14445


In [14]:
MSW_2023 = pd.read_excel("/content/przemoc-domowa-2023.xlsx", skiprows=3,names =["1","2","nazwa","suma"])
MSW_2023=MSW_2023.iloc[[0],[2,3]]
MSW_2023["rok"]=2023
MSW_2022 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2022",names =["1","nazwa","suma"])
MSW_2022=MSW_2022.iloc[[0],[1,2]]
MSW_2022["rok"]=2022
MSW_2021 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2021",names =["1","nazwa","suma"])
MSW_2021=MSW_2021.iloc[[0],[1,2]]
MSW_2021["rok"]=2021
MSW_2020 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2020",names =["1","nazwa","suma"])
MSW_2020=MSW_2020.iloc[[0],[1,2]]
MSW_2020["rok"]=2020
MSW_2019 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2019",names =["1","nazwa","suma"])
MSW_2019=MSW_2019.iloc[[0],[1,2]]
MSW_2019["rok"]=2019
MSW_2018 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2018",names =["1","nazwa","suma"])
MSW_2018=MSW_2018.iloc[[0],[1,2]]
MSW_2018["rok"]=2018
MSW_2017 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2017",names =["1","nazwa","suma"])
MSW_2017=MSW_2017.iloc[[0],[1,2]]
MSW_2017["rok"]=2017
MSW_2016 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2016",names =["1","nazwa","suma"])
MSW_2016=MSW_2016.iloc[[0],[1,2]]
MSW_2016["rok"]=2016
MSW_2015 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2015",names =["1","nazwa","suma"])
MSW_2015=MSW_2015.iloc[[0],[1,2]]
MSW_2015["rok"]=2015
MSW_2014 =pd.read_excel("/content/przemoc-w-rodzinie-2012-2022.xlsx",sheet_name="2014",names =["1","nazwa","suma"])
MSW_2014=MSW_2014.iloc[[0],[1,2]]
MSW_2014["rok"]=2014

FileNotFoundError: [Errno 2] No such file or directory: '/content/przemoc-domowa-2023.xlsx'

In [15]:
def polacz(*datasets):


    return pd.concat(datasets, ignore_index=True)


In [16]:
MSW_2014_2023 = polacz(MSW_2023
   ,MSW_2022,MSW_2021,
MSW_2020, MSW_2019,
MSW_2018,MSW_2017,
MSW_2016,MSW_2015,MSW_2014

)
MSW_2014_2023["suma"]=MSW_2014_2023["suma"].astype(int)
MSW_2014_2023

NameError: name 'MSW_2023' is not defined

In [17]:
# MSW_2014_2023.to_excel("//content/drive/MyDrive/praca_licencjacka/MSW_2014_2023.xlsx", sheet_name= "MSW_2014_2023" ,index=False)

In [18]:
pivot_typy_jednostek_liczba_proc_nk_a_suma.iloc[0:7, 3] = [77808, 75495, 73531, 75662, 73153, 74313, 72601]

In [19]:
pivot_typy_jednostek_liczba_proc_nk_a_suma

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,3151,77808,<NA>,9893,562,13606
2015,<NA>,<NA>,2972,75495,<NA>,9399,372,12743
2016,<NA>,<NA>,3082,73531,<NA>,8344,363,11789
2017,<NA>,<NA>,4775,75662,<NA>,8537,355,13667
2018,<NA>,<NA>,3069,73153,<NA>,7803,305,11177
2019,<NA>,<NA>,3201,74313,<NA>,8559,323,12083
2020,<NA>,<NA>,2646,72601,<NA>,7194,225,10065
2021,3198,33,16295,64250,1138,43869,<NA>,60164
2022,2838,42,3320,61645,2918,10797,328,14445


In [20]:
wystapienia = dane.groupby(["rok", "typ_jednostki"])["rodziny_wszczete"].count()
suma = dane.groupby(["rok", "typ_jednostki"])["rodziny_wszczete"].sum()

typy_jednostki_liczba_rodzin = pd.DataFrame({
    "ilosc_wystapien": wystapienia,
    "suma": suma
}).reset_index()

In [21]:
pivot_typy_jednostek_liczba_rodzin_ilosc_wystapien = typy_jednostki_liczba_rodzin.pivot(index="rok", columns="typ_jednostki", values="ilosc_wystapien")
pivot_typy_jednostek_liczba_rodzin_ilosc_wystapien = pivot_typy_jednostek_liczba_rodzin_ilosc_wystapien.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_rodzin_ilosc_wystapien

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,65,<NA>,<NA>,2432,0,16
2015,<NA>,<NA>,65,<NA>,<NA>,2431,0,16
2016,<NA>,<NA>,65,<NA>,<NA>,2431,0,16
2017,<NA>,<NA>,66,<NA>,<NA>,2430,0,16
2018,<NA>,<NA>,66,<NA>,<NA>,2430,0,16
2019,<NA>,<NA>,66,<NA>,<NA>,2429,0,16
2020,<NA>,<NA>,66,<NA>,<NA>,2429,0,16
2021,0,0,66,0,0,2429,<NA>,16
2022,0,0,66,0,0,2429,314,16


In [22]:
pivot_typy_jednostek_liczba_rodzin_suma = typy_jednostki_liczba_rodzin.pivot(index="rok", columns="typ_jednostki", values="suma")
pivot_typy_jednostek_liczba_rodzin_suma = pivot_typy_jednostek_liczba_rodzin_suma.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_rodzin_suma

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,18857,<NA>,<NA>,53835,0,72692
2015,<NA>,<NA>,20105,<NA>,<NA>,54172,0,74277
2016,<NA>,<NA>,19529,<NA>,<NA>,52817,0,72346
2017,<NA>,<NA>,20132,<NA>,<NA>,54718,0,74850
2018,<NA>,<NA>,20262,<NA>,<NA>,53498,0,73760
2019,<NA>,<NA>,21045,<NA>,<NA>,54189,0,75234
2020,<NA>,<NA>,19163,<NA>,<NA>,52556,0,71719
2021,0,0,30081,0,0,71211,<NA>,101292
2022,0,0,28577,0,0,63727,485,92789


In [23]:
wystapienia = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_ogol"].count()
suma = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_ogol"].sum()

typy_jednostki_liczba_osob_ogol = pd.DataFrame({
    "ilosc_wystapien": wystapienia,
    "suma": suma
}).reset_index()

In [24]:
pivot_typy_jednostek_liczba_osob_ogol_ilosc_wystapien = typy_jednostki_liczba_osob_ogol.pivot(index="rok", columns="typ_jednostki", values="ilosc_wystapien")
pivot_typy_jednostek_liczba_osob_ogol_ilosc_wystapien = pivot_typy_jednostek_liczba_osob_ogol_ilosc_wystapien.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_osob_ogol_ilosc_wystapien

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,65,<NA>,<NA>,2432,0,16
2015,<NA>,<NA>,65,<NA>,<NA>,2431,0,16
2016,<NA>,<NA>,65,<NA>,<NA>,2431,0,16
2017,<NA>,<NA>,66,<NA>,<NA>,2430,0,16
2018,<NA>,<NA>,66,<NA>,<NA>,2430,0,16
2019,<NA>,<NA>,66,<NA>,<NA>,2429,0,16
2020,<NA>,<NA>,66,<NA>,<NA>,2429,0,16
2021,0,0,66,0,0,2429,<NA>,16
2022,0,0,66,0,0,2429,0,16


In [25]:
pivot_typy_jednostek_liczba_osob_ogol_suma = typy_jednostki_liczba_osob_ogol.pivot(index="rok", columns="typ_jednostki", values="suma")
pivot_typy_jednostek_liczba_osob_ogol_suma = pivot_typy_jednostek_liczba_osob_ogol_suma.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_osob_ogol_suma

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,25108,<NA>,<NA>,121339,0,146447
2015,<NA>,<NA>,13763,<NA>,<NA>,135333,0,149096
2016,<NA>,<NA>,16729,<NA>,<NA>,145367,0,162096
2017,<NA>,<NA>,19169,<NA>,<NA>,141586,0,160755
2018,<NA>,<NA>,18725,<NA>,<NA>,143769,0,162494
2019,<NA>,<NA>,21362,<NA>,<NA>,142758,0,164120
2020,<NA>,<NA>,21537,<NA>,<NA>,134936,0,156473
2021,0,0,43480,0,0,118095,<NA>,161575
2022,0,0,39609,0,0,100478,0,140087


In [30]:

wystapienia = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_kobiety"].count()
suma = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_kobiety"].sum()

typy_jednostki_liczba_osob_kobiety = pd.DataFrame({
    "ilosc_wystapien": wystapienia,
    "suma": suma
}).reset_index()

In [31]:
pivot_typy_jednostek_liczba_osob_kobiety_suma = typy_jednostki_liczba_osob_kobiety.pivot(index="rok", columns="typ_jednostki", values="suma")
pivot_typy_jednostek_liczba_osob_kobiety_suma = pivot_typy_jednostek_liczba_osob_kobiety_suma.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_osob_kobiety_suma

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,0,<NA>,<NA>,0,0,0
2015,<NA>,<NA>,9641,<NA>,<NA>,77819,0,87460
2016,<NA>,<NA>,10394,<NA>,<NA>,77712,0,88106
2017,<NA>,<NA>,11327,<NA>,<NA>,76354,0,87681
2018,<NA>,<NA>,10540,<NA>,<NA>,75741,0,86281
2019,<NA>,<NA>,11964,<NA>,<NA>,75157,0,87121
2020,<NA>,<NA>,11635,<NA>,<NA>,71008,0,82643
2021,0,0,24146,0,0,62849,<NA>,86995
2022,0,0,21518,0,0,53994,0,75512


In [33]:
wystapienia = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_mezczyzni"].count()
suma = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_mezczyzni"].sum()

typy_jednostki_liczba_mezczyzni_ogol = pd.DataFrame({
    "ilosc_wystapien": wystapienia,
    "suma": suma
}).reset_index()

In [34]:
pivot_typy_jednostek_liczba_mezczyzni_ogol_suma = typy_jednostki_liczba_mezczyzni_ogol.pivot(index="rok", columns="typ_jednostki", values="suma")
pivot_typy_jednostek_liczba_mezczyzni_ogol_suma = pivot_typy_jednostek_liczba_mezczyzni_ogol_suma.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_mezczyzni_ogol_suma

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,0,<NA>,<NA>,0,0,0
2015,<NA>,<NA>,1770,<NA>,<NA>,24706,0,26476
2016,<NA>,<NA>,3311,<NA>,<NA>,27254,0,30565
2017,<NA>,<NA>,3888,<NA>,<NA>,23056,0,26944
2018,<NA>,<NA>,3841,<NA>,<NA>,24306,0,28147
2019,<NA>,<NA>,4410,<NA>,<NA>,26801,0,31211
2020,<NA>,<NA>,4367,<NA>,<NA>,23609,0,27976
2021,0,0,5440,0,0,17566,<NA>,23006
2022,0,0,3891,0,0,13010,0,16901


In [36]:
wystapienia = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_dzieci"].count()
suma = dane.groupby(["rok", "typ_jednostki"])["liczba_osob_dzieci"].sum()

typy_jednostki_liczba_dzieci_ogol = pd.DataFrame({
    "ilosc_wystapien": wystapienia,
    "suma": suma
}).reset_index()

In [37]:
pivot_typy_jednostek_liczba_dzieci_ogol_suma = typy_jednostki_liczba_dzieci_ogol.pivot(index="rok", columns="typ_jednostki", values="suma")
pivot_typy_jednostek_liczba_dzieci_ogol_suma = pivot_typy_jednostek_liczba_dzieci_ogol_suma.apply(pd.to_numeric, errors='coerce').astype('Int64')
pivot_typy_jednostek_liczba_dzieci_ogol_suma

typ_jednostki,MEiN,MKiDN,MOPR,MSW,MZ,OPS,PCPR,WPS
rok,,,,,,,,
2014,<NA>,<NA>,0,<NA>,<NA>,0,0,0
2015,<NA>,<NA>,2352,<NA>,<NA>,32808,0,35160
2016,<NA>,<NA>,3024,<NA>,<NA>,40401,0,43425
2017,<NA>,<NA>,3954,<NA>,<NA>,42176,0,46130
2018,<NA>,<NA>,4344,<NA>,<NA>,43722,0,48066
2019,<NA>,<NA>,4988,<NA>,<NA>,40800,0,45788
2020,<NA>,<NA>,5535,<NA>,<NA>,40319,0,45854
2021,0,0,13894,0,0,37680,<NA>,51574
2022,0,0,14200,0,0,33474,0,47674
